In [1]:
!pip install pytorch-nlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.4 MB/s 


In [2]:
from torchnlp.datasets import imdb_dataset
from tqdm import tqdm
import re

In [3]:
train = imdb_dataset(train=True)
test = imdb_dataset(test=True)

aclImdb_v1.tar.gz: 84.1MB [00:10, 8.19MB/s]                            


In [4]:
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
def tokenizing(data, mode=0):
    if mode==1:
        text = ' '.join([s["text"] for s in data])
    else:
        text = data
    text = text.replace('<br /><br />',' ')
    text = text.replace('\x85','')
    text = text.lower()
    tokenized_sentence = re.compile('[.!?()]').split(text)
    tokenized_corpus = [None]*len(tokenized_sentence)
    if mode==1:
        for i in tqdm(range(len(tokenized_sentence))):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    else:
        for i in range(len(tokenized_sentence)):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    return tokenized_corpus

In [6]:
tokenized_corpus = tokenizing(train,mode=1)

100%|██████████| 439413/439413 [00:51<00:00, 8460.80it/s] 


In [7]:
from gensim.models import Word2Vec
import multiprocessing

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(sentences=tokenized_corpus, progress_per=10000)

In [20]:
import gensim.downloader as api
w2v_model = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [21]:
w2v_model.train(sentences=tokenized_corpus, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
# model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")

AttributeError: ignored

In [11]:
w2v_model.wv.most_similar('computer', topn=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('camera', 0.907833456993103),
 ('cell', 0.891890287399292),
 ('server', 0.8744666576385498),
 ('device', 0.869352400302887),
 ('wifi', 0.8631256818771362),
 ('screen', 0.8621907234191895),
 ('app', 0.8615544438362122),
 ('case', 0.8587921857833862),
 ('remote', 0.8583616018295288),
 ('file', 0.8575270771980286)]

In [12]:
import pickle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pickle.dump(w2v_model,open('/content/drive/MyDrive/NLP/w2v_model_gensim.p','wb'))

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/NLP/w2v_model_gensim.p','rb'))

In [ ]:
model.wv.most_similar('woman', topn=10)

[('man', 0.6788017153739929),
 ('girl', 0.6409904956817627),
 ('she', 0.505253255367279),
 ('guy', 0.4965588450431824),
 ('women', 0.47835952043533325),
 ('her', 0.45994827151298523),
 ('boy', 0.4510478079319),
 ('young', 0.437908411026001),
 ('hooker', 0.43279820680618286),
 ('loretta', 0.4306209981441498)]

In [22]:
model = w2v_model

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
  def __init__(self, x, y, z):
    self.data = x
    self.labels = y
    self.lengths = z

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index], self.lengths[index]

In [24]:
import random
random.shuffle(train)
def check_vocab(x):
    if x in list(model.wv.vocab):
        return True
    else:
        return False

vector_size = 300
x = []
y = []
z = []
for a in tqdm(train[0:1250]):
  if a!=[]:
    sentiment = int(a['sentiment']=='pos')
    for sentence in tokenizing(a['text']):
      sentence = list(filter(check_vocab, sentence))
      if len(sentence) >= 5:
        x.append(torch.reshape(torch.tensor([model.wv[word] for word in sentence]), (1,vector_size, len(sentence))))
        y.append([sentiment])
        z.append(len(sentence))
train_dataset = MyDataset(x,y,z)

  0%|          | 0/1250 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
 12%|█▏        | 155/1250 [05:38<39:48,  2.18s/it]


KeyboardInterrupt: ignored

In [27]:
train_dataset = MyDataset(x,y,z)

In [25]:
def custom_collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    _, labels, lengths = zip(*data)
    max_len = max(lengths)
    fix_size = data[0][0].size(1)
    features = torch.zeros((len(data), 1, fix_size, max_len))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
      k, j = data[i][0][0].size(0), data[i][0][0].size(1)
      features[i][0] = torch.cat((data[i][0][0], torch.zeros((k, max_len - j))),1)

    return features.float(), labels.long(), lengths.long()

In [ ]:
batch = [train_dataset[0],train_dataset[0]]
a,b,c = torch.utils.data.default_collate(batch)
a.size()

torch.Size([2, 1, 300, 9])

In [ ]:
features, labels, lengths = custom_collate_fn(batch)

In [ ]:
features.size()

torch.Size([1, 5, 300, 36])

In [ ]:
data = batch
_, labels, lengths = zip(*data)
max_len = max(lengths)
fix_size = data[0][0].size(1)
features = torch.zeros((len(data),1, fix_size, max_len))
labels = torch.tensor(labels)
lengths = torch.tensor(lengths)

In [ ]:
features.size()

torch.Size([2, 1, 300, 9])

In [ ]:
labels

tensor([1, 1, 1, 1, 1])

In [ ]:
lengths

tensor([ 9, 10, 16, 36, 14])

In [ ]:
for i in range(len(data)):
    k, j = data[i][0][0].size(0), data[i][0][0].size(1)
    features[i][0] = torch.cat((data[i][0][0], torch.zeros((k, max_len - j))),1)

In [ ]:
features[1][0].size()

torch.Size([300, 9])

In [ ]:
torch.zeros((300, 12 - 8)).size()

torch.Size([300, 4])

In [ ]:
data[1][0][0].size()

torch.Size([300, 10])

In [ ]:
data[i][0].size(1)

12

In [ ]:
torch.cat((data[1][0],torch.zeros((300, 12 - 8))),1).size()

torch.Size([300, 12])

In [28]:
# del x
# del y
# del train
train_loader = torch.utils.data.DataLoader(train_dataset, collate_fn=custom_collate_fn, batch_size=50, shuffle=True, num_workers=2)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=2)
# train_loader = train_dataset

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,2))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,3))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,4))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 200,
                                kernel_size = (vector_size,5))
        self.fc1 = nn.Linear(800, 1)
        self.dropout = nn.Dropout(0.50)
    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x1 = F.max_pool2d(x1, (1,x1.size()[-1]))
        x2 = F.relu(self.conv2(x))
        x2 = F.max_pool2d(x2, (1,x2.size()[-1]))
        x3 = F.relu(self.conv3(x))
        x3 = F.max_pool2d(x3, (1,x3.size()[-1]))
        x4 = F.relu(self.conv4(x))
        x4 = F.max_pool2d(x4, (1,x4.size()[-1]))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),1),start_dim=1)
        x = self.fc1(self.dropout(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# x = train_dataset.data[4]
x = inputs
a = F.relu(net.conv1(x.to(device)))
b = F.relu(net.conv2(x.to(device)))
c = F.relu(net.conv3(x.to(device)))
#torch.transpose(torch.cat((torch.max(a,2).values,torch.max(b,2).values,torch.max(c,2).values)),0,1)
#torch.flatten(torch.cat((torch.max(a,3).values,torch.max(b,3).values,torch.max(c,3).values)))
#torch.flatten(torch.cat((F.max_pool2d(a, (1,a.size()[-1])),F.max_pool2d(a, (1,a.size()[-1])))))

In [ ]:
F.max_pool2d(a, (1,a.size()[-1])).size()

torch.Size([50, 2, 1, 1])

In [ ]:
torch.cat((F.max_pool2d(a, (1,a.size()[-1])),F.max_pool2d(a, (1,a.size()[-1]))),1).size()

torch.Size([50, 4, 1, 1])

In [ ]:
torch.flatten(torch.cat((F.max_pool2d(a, (1,a.size()[-1])),F.max_pool2d(a, (1,a.size()[-1]))),1),start_dim=1).size()

torch.Size([50, 4])

In [ ]:
torch.flatten(net.fc1(torch.flatten(torch.cat((F.max_pool2d(a, (1,a.size()[-1])),F.max_pool2d(a, (1,a.size()[-1])),F.max_pool2d(a, (1,a.size()[-1]))),1),start_dim=1))).size()

torch.Size([50])

In [ ]:
a.size()

torch.Size([50, 2, 1, 59])

In [ ]:
x.size()

torch.Size([50, 1, 300, 60])

In [ ]:
import time

In [ ]:
start_time = time.time()
for i in range(100000):
  torch.flatten(torch.cat((torch.max(b,2).values,torch.max(b,2).values,torch.max(c,2).values)))
time.time() - start_time

5.425304174423218

In [ ]:
start_time = time.time()
for i in range(100000):
  torch.transpose(torch.cat((torch.max(b,2).values,torch.max(b,2).values,torch.max(c,2).values)),0,1)
time.time() - start_time

5.314195156097412

In [ ]:
t = train_dataset.data[3]
net(t.to(device))

tensor([0.5586], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [ ]:
t.size()

torch.Size([1, 300, 36])

In [ ]:
inputs.size()

torch.Size([50, 1, 300, 60])

In [ ]:
net(inputs.to(device)).size()

torch.Size([50, 1])

In [30]:
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, output, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        # _, predicted = torch.max(output.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        # self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total

In [31]:
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.01)
net = net.float()
net.train()
accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs.float())
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels.float())
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * accuracy.compute()))

print('Finished Training')


Starting epoch 1


loss: 0.01842|acc: 53.58%: 100%|██████████| 41/41 [00:08<00:00,  4.97it/s]


Starting epoch 2



loss: 0.01254|acc: 72.18%: 100%|██████████| 41/41 [00:00<00:00, 57.25it/s]


Starting epoch 3


...


Starting epoch 27



loss: 0.00881|acc: 96.76%: 100%|██████████| 41/41 [00:00<00:00, 55.82it/s]



Starting epoch 28


loss: 0.01055|acc: 95.78%: 100%|██████████| 41/41 [00:00<00:00, 56.78it/s]


KeyboardInterrupt: ignored

In [ ]:
pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
net = pickle.load(open('/content/drive/MyDrive/NLP/imdb_sentiment.p','rb'))

In [32]:
random.shuffle(test)
vector_size = 300
x = []
y = []
z = []
for a in tqdm(test[0:12500]):
  if a!=[]:
    sentiment = int(a['sentiment']=='pos')
    for sentence in tokenizing(a['text']):
      sentence = list(filter(check_vocab, sentence))
      if len(sentence) >= 5:
        x.append(torch.reshape(torch.tensor([model.wv[word] for word in sentence]), (1,vector_size, len(sentence))))
        y.append([sentiment])
        z.append(len(sentence))
val_dataset = MyDataset(x,y,z)
val_loader = torch.utils.data.DataLoader(val_dataset, collate_fn=custom_collate_fn, batch_size=50, shuffle=True, num_workers=2)

  0%|          | 0/12500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
  0%|          | 27/12500 [00:54<7:00:09,  2.02s/it]


KeyboardInterrupt: ignored

In [33]:
val_dataset = MyDataset(x,y,z)
val_loader = torch.utils.data.DataLoader(val_dataset, collate_fn=custom_collate_fn, batch_size=50, shuffle=True, num_workers=2)

In [ ]:
net.eval()
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, output, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        # _, predicted = torch.max(output.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        # self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total

accuracy = Accuracy()

accuracy.reset()
# Gradients are calculated on the forward pass for every iteration.
# As we do not need gradients now, we can disable the calculation.
with torch.no_grad():
    for data in tqdm(train_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)

print("Training Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 325/325 [00:28<00:00, 11.24it/s]

Training Accuracy: 98.69%


In [34]:
net.eval()
accuracy = Accuracy()
accuracy.reset()        
with torch.no_grad():
    for data in tqdm(val_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)
        
print("\nTesting Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 6/6 [00:00<00:00, 24.42it/s]


Testing Accuracy: 53.47%


In [ ]:
((outputs.data > 0.5) == labels).sum().item()

289

In [ ]:
labels.size(0)

512